# Run the pipeline with Nexus/Luigi/bbp-workflow

This notebook provides an example of how to run the pipeline using the Nexus access point, Luigi and bbp-workflow. For more details, please refer to the [README](./README.md).

In [1]:
import logging
from bluepyemodel.model.model_configurator import ModelConfigurator
from bluepyemodel.emodel_pipeline.emodel_pipeline import EModel_pipeline
from bluepyemodel.emodel_pipeline.emodel_settings import EModelPipelineSettings
from bluepyemodel.efeatures_extraction.targets_configurator import TargetsConfigurator

In [2]:
emodel = "L5_TPC"
etype = "cAC"
mtype = None
ttype = None
iteration_tag = "XXXX-XX-XX"
species = "mouse"
brain_region = "SSCX"
iteration="test-bpem-nexus"
morphology = "C060114A5"
morphology_format = "asc"

In [3]:
# Nexus settings
nexus_project = "" # specify the nexus project
nexus_organisation = "" # specify the nexus organisation
nexus_endpoint = "prod"
forge_path = "./forge.yml"
forge_ontology_path = "./nsg.yml"
legacy_conf_json_file = "./../L5PC/config/params/pyr.json" # specify the path to the json file containing the model configuration if the model is not configured from gene data

In [4]:
logger = logging.getLogger()
logging.basicConfig(
    level=logging.INFO,
    handlers=[logging.StreamHandler()],
)

In [ ]:
pipeline = EModel_pipeline(
    emodel=emodel,
    etype=etype,
    mtype=mtype,
    ttype=ttype,
    iteration_tag=iteration,
    species=species,
    brain_region=brain_region,
    forge_path=forge_path,
    forge_ontology_path=forge_ontology_path,
    nexus_organisation=nexus_organisation,
    nexus_project=nexus_project,
    nexus_endpoint=nexus_endpoint,
    use_ipyparallel=True,
    use_multiprocessing=False,
    data_access_point="nexus",
)

To begin, we need to initialize the ``TargetsConfigurator`` using the provided ``targets.py`` file. This file contains predefined configurations for targets and file metadata needed for the feature extraction process, and it can be customized as needed.

In [ ]:
from targets import filenames, ecodes_metadata, targets, protocols_rheobase

files_metadata = []
for filename in filenames:
    files_metadata.append({"cell_name": filename, "ecodes": ecodes_metadata})

targets_formated = []
for ecode in targets:
    for amplitude in targets[ecode]["amplitudes"]:
        for efeature in targets[ecode]["efeatures"]:
            targets_formated.append(
                {
                    "efeature": efeature,
                    "protocol": ecode,
                    "amplitude": amplitude,
                    "tolerance": 10.0,
                }
            )

configurator = TargetsConfigurator(pipeline.access_point)
configurator.new_configuration(files_metadata, targets_formated, protocols_rheobase)
configurator.save_configuration()
print(configurator.configuration.as_dict())

Next, we need to store the ``EModelPipelineSettings`` (EMPS). The following cell creates an instance of EMPS with various parameters, which can be customized according to your needs:

In [9]:
pipeline_settings = EModelPipelineSettings(
        stochasticity=False,
        optimiser="SO-CMA",
        optimisation_params={"offspring_size": 20},
        optimisation_timeout=600.0,
        threshold_efeature_std=0.1,
        max_ngen=100,
        validation_threshold=10.0,
        plot_extraction=True,
        plot_optimisation=True,
        compile_mechanisms=True,
        name_Rin_protocol="IV_-40",
        name_rmp_protocol="IV_0",
        efel_settings={
            "strict_stiminterval": True,
            "Threshold": -20,
            "interp_step": 0.025,
        },
        strict_holding_bounds=True,
        validation_protocols=["IDhyperpol_150"],
        morph_modifiers=[],
        n_model=3,
        optimisation_batch_size=3,
        max_n_batch=5,
        name_gene_map="Mouse_met_types_ion_channel_expression",
    )

pipeline.access_point.store_pipeline_settings(pipeline_settings)

<action> _deprecate_one
<succeeded> True
<action> _register_one
<succeeded> True


In the next cell, we will use ``ICSelector`` to generate the ``EModelConfiguration`` (EMC) based on gene data. This process will use the provided ``etype``, ``mtype``, and ``ttype`` to select appropriate ion channel resources from Nexus.

In [ ]:
pipeline.configure_model(morphology_name=morphology, morphology_format=morphology_format, use_gene_data=True)

Before running the Luigi pipeline, we need to configure the [launch script](./launch_luigi.sh) and the [Luigi configuration script](./luigi.cfg). See sections 2 and 4, respectively, in the following [README](./README.md) for more details.

Once the coniguration is done, we can run the Luigi pipeline in the terminal, by following these steps:



Activate the virtual environment:

``source myvenv/bin/activate``

Initialize Kerberos authentication: 

``kinit``


Launch the Luigi pipeline:

``./launch_luigi.sh``